<h1>Toronto Clustering</h1>
<h2>Part 1</h2>

3 ) Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [48]:
! pip install beautifulsoup4
! pip install lxml
! pip install requests

In [115]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [127]:
#import library to query webpage of interest
from urllib.request import urlopen

#specifying page of interest
wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#save the HTML of the site within the page variable
page = urlopen(wiki)

#import library to parse HTML from page
from bs4 import BeautifulSoup

#parse data from "page" and save to new variable "soup"
soup = BeautifulSoup(page)

In [128]:
#pinpointing the location of the table and its contents
first_table = soup.find('table',{'class':'wikitable sortable'})

#creating lists for each of the columns I know to be in my table.
A=[]
B=[]
C=[]


#utilizing HTML tags for rows <tr> and elements <td> to iterate through each row of data and append data elements to their appropriate lists:

for row in first_table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3 and cells[1].find(text=True) != "Not assigned": #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        
#create dataframe with values populated and strip off \n       
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']= B
df['Neighbourhood']= [s.replace("\n",'') for s in C]
df.head(7)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned


In [129]:
#replace Neighborhood with Borough if it's "Not assigned"

df['Neighbourhood'] = np.where(df['Neighbourhood'] == "Not assigned", df['Borough'], df['Neighbourhood'])
df.head(7)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park


In [130]:
#group by postal code 
df_grouped = (df.groupby('PostalCode')
    .agg({'Neighbourhood': ','.join, 'Borough': 'first'})
    .reset_index()
    .reindex(columns=df.columns))
df_grouped.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [131]:
#return shape of dataframe!
df_grouped.shape

(103, 3)

<h2>Part 2 </h2>

In [ ]:
Use the Geocoder package of csv file to create the dataframe

In [138]:
#import the csv
df_survey = pd.read_csv('http://cocl.us/Geospatial_data')
df_survey.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_survey.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [139]:
#create merged dataframe
df_final = pd.merge(df_grouped, df_survey, on= 'PostalCode')
df_final.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


(103, 3)